### classes and functions

In [1]:
class patient:
    def __init__(self, filepath, input_fileName, lengthPtID):
        ptID = input_fileName[:lengthPtID]
        self.ptID = ptID
        self.fileName = input_fileName
        #find the demographics in one of two sheets
        # directory = os.getcwd()
        file_adds = filepath + '/' + input_fileName
        file = pd.ExcelFile(file_adds)
        sheet_Names = file.sheet_names
        if 'Cancer_Registery Data' in sheet_Names:
            #do the thing for getting patient info
            df = pd.read_excel(input_fileName, sheet_name= 'Cancer_Registery Data')
            self.ptDemographics = df
            self.amyloid_status = df.at[0,"Amyloid Status"]
            self.survival = df.at[0,"Survival Time (Months)"]
            self.ageDx = df.at[0,"Age At Diagnosis"] 
            self.sex = df.at[0,"Gender Cancer Registry"]
            self.vitalStatus = df.at[0, "Vital Status Cancer Registry"]
            self.dx = df.at[0, "Histology"]

        elif '30 Cerner Patients ' in sheet_Names:
            #do the thing for getting patient info
            df = pd.read_excel(input_fileName, sheet_name= '30 Cerner Patients ')
            self.ptDemographics = df
            self.amyloid_status = df.at[0,"Amyloid Status"]
            self.survival = df.at[0,"Survival Time (months)"]
            self.ageDx = df.at[0,"Age At Diagnosis"] 
            self.sex = df.at[0,"Gender Cerner"]
            self.vitalStatus = df.at[0, "Vital Status (Cerner)"]
            self.dx = df.at[0, "Primary Site"]


        else: print("no patient identifying information")

        #now iterate through the list of sheets to import the patient data 
        if 'Labs' in sheet_Names:
            self.labsData = pd.read_excel(input_fileName, sheet_name= 'Labs')
        else: self.labsData = False
        
        if 'amyloid in subject' in sheet_Names:
            self.echoData = pd.read_excel(input_fileName, sheet_name= 'amyloid in subject')
        else: self.echoData = False

        if 'ChemoTx' in sheet_Names:
            self.ChemoTx = pd.read_excel(input_fileName, sheet_name= 'ChemoTx')
        else: self.ChemoTx = False

        if 'Hematologic Tx' in sheet_Names:
            self.HemeTx = pd.read_excel(input_fileName, sheet_name= 'Hematologic Tx')
        else: self.HemeTx = False

        if 'Immuno Tx' in sheet_Names:
            self.ImmunoTx = pd.read_excel(input_fileName, sheet_name= 'Immuno Tx')
        else: self.ImmunoTx = False

        if 'Radiation Tx' in sheet_Names:
            self.RadTx = pd.read_excel(input_fileName, sheet_name= 'Radiation Tx')
        else: self.RadTx = False

        if 'Other Tx' in sheet_Names:
            self.OtherTx = pd.read_excel(input_fileName, sheet_name = 'Other Tx')
        else: self.OtherTx = False

        if 'Physican_Notes' in sheet_Names:
            self.MDnotes = pd.read_excel(input_fileName, sheet_name= 'Physican_Notes')
        else: self.MDnotes = False

        #to create a callable list of treatments recieved for which we have data
        tx_sheets = ['ChemoTx', 'Hematologic Tx', 'Immuno Tx']
        treatmentTypesRecieved = []

        for possible in tx_sheets:
        
            if possible in sheet_Names:
                treatmentTypesRecieved.append(possible)
        self.txRecieved = treatmentTypesRecieved

def FilterList(list, keyWords_primary, keyWords_secondary = ["Empty"], omit = ["Empty"]):
    #store the filtered result
    filteredList = []

    if keyWords_secondary == ["Empty"]:
        for c in list: #loop the columns
            for buzz in keyWords_primary: #loop the key words
                if buzz in c: #if the column contains the key word
                    filteredList.append(c) #add the column to the list
                    break #do not continue testing primary key words for this column
    
    else: 
        for c in list: #loop the columns
            for buzz in keyWords_primary: #loop the key words
                if buzz in c: #if the column contains the key word
                    for secondary in keyWords_secondary:
                        if secondary in c:
                            filteredList.append(c) #add the column to the list
                            break #has been added to list based on passing the seondary 
                        else:
                            pass
                    break #do not continue to check primary key words for this column
                else:
                    pass
                
    if omit != ["Empty"]: 
        for x in omit: 
            for a in filteredList:
                if x in a:
                    filteredList.remove(a)
    
    #by iterating through the column names first, we keep the order of the columns

    return filteredList 

#createBinary function
def createBinary(df, listOfLabs, timeColumn): 
    #list of labs needs to match exactly.  
    # Filter the dataframe before inputting
    #for boolean
    df_toBool = df[listOfLabs]
    df_bool = df_toBool.notna()
    df_asint = df_bool.astype(int)
    #add column back for time 
    df_asint.insert(0, timeColumn, df[timeColumn])
    return df_asint

def createBinarySum(df, listOfLabs, timeColumn, defaxis):
    #list of labs needs to match exactly.  
    # Filter the dataframe before inputting
    #for boolean
    df_toBool = df[listOfLabs]
    df_bool = df_toBool.notna()
    df_asint = df_bool.astype(int)
    #add column back for time 
    # df_asint.insert(0, timeColumn, df[timeColumn])
    #sum across the row
    #defaxis = 0 means the columns are preserved 
    #defaxis =1 means the rows are preserved
    df_new_sum = df_asint.sum(axis = defaxis)
    return df_new_sum

#get list of patients from directory 
def GetListOfPatientsFromDirectory (directory, fileExtension, fileCommon, lengthID):
    #get the directory 
    listFileNames = os.listdir(directory)
    outputList = []
    #loop through list of fileNames: 
    for file in listFileNames:
        #id if the file is patient data
        if fileCommon in file and file.endswith(fileExtension):
            #trim the file name to the pt ID
            ptID = file[:lengthID]
            outputList.append(ptID)
    return outputList


def GetListofPTfiles(directory, fileExtension, fileCommon):
    listFileNames = os.listdir(directory)
    outListFiles = []
    #loop through file names
    for file in listFileNames:
        if fileCommon in file and file.endswith(fileExtension):
            outListFiles.append(file)
    return outListFiles
def getNormalValue(filteredDataColumn, skipValue = 6666):
    test = filteredDataColumn
    #input is a series
    
    for entry in test:
        if entry != skipValue:
            outputValue = entry
                # print("this is output: ", outputValue)
                #set x to True
            break
        else:
            # print("entry is blank")
            pass
    return outputValue

#get amyloid status dictionary inputs, sort ptIDs to lists based on amyloid status 
def getListSortedByAmyloid(amyloidDictionary, listPositive, listNegative, listOther,
                           statusPositive = "yes", statusNegative = "no"):
    for pt in amyloidDictionary.keys():
        status = amyloidDictionary[pt]
        if status == statusPositive:
            listPositive.append(pt)
        elif status == statusNegative:
            listNegative.append(pt)
        else:
            listOther.append(pt)
    #does not check for double listed patient IDs
    
#function for masking a dataframe 
def patientSectionOfFrame(start_frame, column_toMask, ptID):
    #ptID is a row parameter in the column_toMask
    df_use = start_frame #copy of the frame

    #first check if sheet has the column of interest
    data_top = list(start_frame.columns) #returns the headers as a list

    #determine of the sheet contains the column of interest
    if data_top.count(column_toMask) > 0: 
        #if true, now find if the patient is present
        searchForPt = start_frame[column_toMask].str.startswith(ptID).sum() #returns the number of instances of the patient
        if searchForPt > 0:
            #patient is present 
            #mask the data for the patient
            df_clean = df_use.dropna(subset= [column_toMask]) #removing NA 
            mask = df_clean[column_toMask].str.startswith(ptID) #masking for the rows where the patient has data
            df_sub_tosave = df_clean[mask] #output dataframe filtered for the patient
            return df_sub_tosave
        else:
            #the patient isn't present
            # return print("The patient " + ptID + " is not in " + sheet_name)
            pass
    else:
        #the column of interest isn't present
        # return print("The column " + column_toMask + " is not in the sheet " + sheet_name)
        pass


def outputToExcel(df_data, fileName_header, fileName_Common, fileName_suffix,
                  parent_dir, folderName, sheetName):
    fileName = fileName_header + fileName_Common + fileName_suffix
    outfile_extension = '.xlsx'
    outfile_boxplts = fileName + outfile_extension
    path_out= os.path.join(parent_dir, folderName, outfile_boxplts)

    if os.path.exists(path_out):
            #if old sheet
            with pd.ExcelWriter(path_out, mode = 'a', if_sheet_exists = 'overlay') as writer:
                    df_data.to_excel(writer, sheet_name = sheetName, index = True)
    else: 
            #new sheet
            with pd.ExcelWriter(path_out) as writer:
                    df_data.to_excel(writer, sheet_name = sheetName, index = True) #if new sheet
    
def outputFiguresPath(fileName_header, fileName_mid, fileName_suff, parent_dir, folderName):
        fileName_header = str(fileName_header)
        fileName_mid = str(fileName_mid)
        fileName_suff = str(fileName_suff)

        #cleaning the input to prevent addition of / to the directory
        fileName_header = fileName_header.replace("/", "-")
        fileName_mid = fileName_mid.replace("/", "-")
        fileName_suff = fileName_suff.replace("/", "-")

        fileName = fileName_header + fileName_mid +fileName_suff
        out_filename = fileName + '.tif'
        new_filepath = os.path.join(parent_dir, folderName, out_filename)
        #the output path can be used to save the figure
        return new_filepath
        # plt.savefig(new_filepath, bbox_inches = 'tight')

def makeFolderPathForData(parent_dir, folderName_header, folderName_common, folderName_suffix):
    #makes a new directory for your files
    #returns the folder name for use in other functions
    folderName = folderName_header + folderName_common + folderName_suffix
    path = os.path.join(parent_dir,folderName)
    if not os.path.exists(path):
        os.mkdir(path)
    return folderName

### libraries and directories

In [2]:
#libraries and dirrectory 
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy.optimize import curve_fit

In [3]:
#location of original data
starting_directory = os.getcwd()

new_dirrectory = 'C:/Users/maega/Documents/3000 PhD/3300_BEAT Labs/Projects/Cardiac-Amyloidosis-Multiple-Myeloma/012 Processed Data/data-2024-06-05'
os.chdir(new_dirrectory)

In [4]:
#file name suffix 
outputSheet_Suffix = "_2024-07-16_v1"
outputSheet_common = "-allLabs-wFill-wTx-Summary"

In [5]:
#output dirrectory
outputDir = "C:/Users/maega/Documents/3000 PhD/3300_BEAT Labs/Projects/Cardiac-Amyloidosis-Multiple-Myeloma/012 Processed Data"
folderNameOut = makeFolderPathForData(parent_dir=outputDir, folderName_header="DFsForSVM", folderName_common="_2024-07-16", folderName_suffix="_v1")

In [6]:
#filler information for fits of 1 point
one_point_STD = 0
one_point_r2 = 1000000
one_point_residual_error = 1000000

In [7]:
#starting code for saving files
    #fileoutput 
    # outputToExcel(df_data = df_parameters_linearRegression, fileName_header= ptID_test, fileName_Common= dataOut_common, fileName_suffix= dataOut_suffix,
    #             parent_dir= patientFitsDir, folderName=patientFitsFolder, sheetName= dataOut_sheetName)


### Collecting the patient information

In [8]:
#importing patient data as dictionaries of dataframes

#build a dictionary of patients and their values 
ptFiles = GetListofPTfiles(new_dirrectory, '.xlsx', 'data')

patientDictAmyloid = {}
patientDictSurvival = {}
patientDictLabs = {}
patientDictEcho = {}
patientDictNotes = {}
patientDictDx = {}
patientDictVitalStatus = {}
patientDictSex = {}
patientDictAge = {}
patientDictChemo = {}
patientDictImmuno = {}
patientDictHeme = {}
patientDictRad = {}
patientDictOther = {}

#loop through the listOfPatients 
for file in ptFiles: 
    pt = patient(new_dirrectory, file, 4)
    patientDictAmyloid[pt.ptID] = pt.amyloid_status
    patientDictSurvival[pt.ptID] = pt.survival
    patientDictLabs[pt.ptID] = pt.labsData
    patientDictEcho[pt.ptID] = pt.echoData
    patientDictNotes[pt.ptID] = pt.MDnotes
    patientDictDx[pt.ptID] = pt.dx
    patientDictVitalStatus[pt.ptID] = pt.vitalStatus
    patientDictSex[pt.ptID] = pt.sex
    patientDictAge[pt.ptID] = pt.ageDx
    patientDictChemo[pt.ptID] = pt.ChemoTx
    patientDictImmuno[pt.ptID] = pt.ImmunoTx
    patientDictHeme[pt.ptID] = pt.HemeTx
    patientDictRad[pt.ptID] = pt.RadTx
    patientDictOther[pt.ptID] = pt.OtherTx

In [9]:
EchoTimeColumn ='time from diagnosis to first echo (months)'
echoDates = {}
for pt in patientDictEcho:
    ptEchoData = patientDictEcho[pt]
    patientEchoDate = ptEchoData[EchoTimeColumn]
    if 'none' not in list(patientEchoDate):
        echoDates[pt] = patientEchoDate

### Collecting the lab columns we need

In [10]:
# columnsToUse = ['Creatinine Result(mg/dL)','Potassium Result(mmol/L)','total CO2 Result','Chloride Result(mmol/L)',
#                 'Calcium Result(mg/dL)','Sodium Result(mmol/L)','Glucose Result','BUN Result(mg/dL)','Platelet Count Result(k/uL)',
#                 'Hemoglobin Result','MCHC Result','MCV Result','MCH Result(pg)','HCT Result','RBC Result','WBC Result(k/uL)',
#                 'Albumin Result(gm/dL)','Neutrophil Result','Monocyte Result','Pct. Neutrophil Result','Pct. Mono Result',
#                 'Total Serum Protein Result','Pct. Immature Gran Auto Result','Nucleated RBC Result','AST Result(U/L)',
#                 'ALT Result(U/L)','Serum Electrophoresis Alpha 1 Result','Serum Electrophoresis Alpha 2 Result',
#                 'Serum Electrophoresis Beta Result','Serum Electrophoresis Gamma Result','Albumin Electrophoresis Result',
#                 'LDH Result(U/L)','Kappa Free Light Chains Result (mg/L)','Lambda Free Light Chains Result (mg/L)',
#                 'Kappa/Lambda Free Light Chain Ratio Result','Beta-2-Microglobulin Result',
#                 'NT Pro BNP (B-Type Natriuretic Peptide)','Troponin-T']


In [11]:
#dictionary for the filtered data and the list of columnNames
#this enables filtering of the patient data as its own block of code
dictOfFilteredLabs = {}

#other input values to set outside of loop 
x_column_name = "RelTime(Days)"
#creating dictionaries for all patients in the dirrectory 

#also collects the column names of interest 

for ptID in (patientDictLabs.keys()):
    #to collect and filter the patient labs into a dictionary
    df_Labs = patientDictLabs[ptID]

    orig_columns = df_Labs.columns

    #filter the patient labs
    columnNames = FilterList(orig_columns, keyWords_primary = ['Result', 'Troponin-T', 'BNP'],  
                                omit = ['Method', 'ABO', 'Dohle', "Toxic Granulation", "D Dimer Result",
                                        "Urine Color Result", "Urine Leukocyte Esterase Result",
                                        "Urine Nitrite Result", "Urinalysis Squamous Epithelial Cells Result (/hpf)",
                                        "(/hpf)", "Hep B", "Hep B Surf Ag Result", "Hep C", "Hep C Ab Screen Result",
                                        "Hypersegmentation Result", "HCG Result (Serum)", "Coombs Test Result",
                                        "HIV Result"])
    # columnNames = FilterList(orig_columns, keyWords_primary = columnsToUse,
    #                          omit = ['Method', 'ABO', 'Dohle', "Toxic Granulation", "D Dimer Result",
    #                                     "Urine Color Result", "Urine Leukocyte Esterase Result",
    #                                     "Urine Nitrite Result", "Urinalysis Squamous Epithelial Cells Result (/hpf)",
    #                                     "(/hpf)", "Hep B", "Hep B Surf Ag Result", "Hep C", "Hep C Ab Screen Result",
    #                                     "Hypersegmentation Result", "HCG Result (Serum)", "Coombs Test Result",
    #                                     "HIV Result", "Urine Glucose Result(mg/dL)"])
    
    
    testNames = [x_column_name]
    testNames.extend(columnNames) #addition of time label


    ptLabs_results= df_Labs[testNames] #new data frame of just results

    #troponin contains strings which mess up plotting 
    df_Labs_results_2 = ptLabs_results
    # for col in columnNames:
    #     df_Labs_results_2[col] = ptLabs_results[col].apply(lambda x: 0 if isinstance(x, str) else x)
    df_Labs_results_2["Troponin-T"] = ptLabs_results["Troponin-T"].apply(lambda x: 0 if isinstance(x, str) else x)
    df_Labs_results_2["NT Pro BNP (B-Type Natriuretic Peptide)"] = ptLabs_results["NT Pro BNP (B-Type Natriuretic Peptide)"].apply(lambda x: 0 if isinstance(x, str) else x)
    df_Labs_results_2['Anisocytosis Result'] = df_Labs_results_2['Anisocytosis Result'].replace({'1+': 1, '2+': 2,'3+': 3})
    df_Labs_results_2['Macrocytes Result'] = df_Labs_results_2['Macrocytes Result'].replace({'1+': 1, '2+': 2,'3+': 3})

    ptLabs_results_2 = df_Labs_results_2
    dictOfFilteredLabs[ptID] = ptLabs_results_2


C:\Users\maega\AppData\Local\Temp\ipykernel_28836\1747423668.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Labs_results_2["Troponin-T"] = ptLabs_results["Troponin-T"].apply(lambda x: 0 if isinstance(x, str) else x)
C:\Users\maega\AppData\Local\Temp\ipykernel_28836\1747423668.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Labs_results_2["NT Pro BNP (B-Type Natriuretic Peptide)"] = ptLabs_results["NT Pro BNP (B-Type Natriuretic Peptide)"].apply(lambda x: 0 if isinstance(x, str) else x)
C:\U

### Make list by amyloid status

In [12]:
#list of patients by amyloid status 

listPositive = []
listNegative = []
listOther = []

getListSortedByAmyloid(patientDictAmyloid, listPositive= listPositive,
                       listNegative=listNegative, listOther=listOther)

#first make a copy of the list
listALKnown = listPositive.copy()
#add the negative patients to the list of positive patients
listALKnown.extend(listNegative) 

In [13]:
#length of groups 
print("Lenght of positive: ", len(listPositive))
print("length of negative: ", len(listNegative))
print("list of unknown: ", len(listOther))

Lenght of positive:  24
length of negative:  24
list of unknown:  19


### Demographic information

In [14]:
dict_dx_codes = {
    "MGUS" : 1, 
    "MM" : 2, 
    "Multiple Myeloma" : 2,
    "MULTIPLE MYELOMA" : 2 
    #other values are going to be set to 0
}

dict_AL_codes = {
    "yes" : 2, 
    "no" : 1, 
    "unk" : 3,
}

dict_sex_codes = {
    "Male" : 0,
    "MALE": 0, 
    "Female" : 1, 
    "FEMALE" : 1,  
}

dict_of_codes = {"AL": dict_AL_codes, 
                 "Dx": dict_dx_codes, 
                 "Sex": dict_sex_codes}

dict_of_data = {"AL": patientDictAmyloid, 
                "Dx": patientDictDx, 
                "Sex": patientDictSex}

dict_encoded_pt_AL_Dx_sex = {}

for ptID in list(patientDictDx.keys()):
    listPtValues = []
    for key in list(dict_of_codes.keys()):
        LocalValue = ''
        for code in dict_of_codes[key]:
            if code in dict_of_data[key][ptID]:
                # print(ptID, " code: ", code)
                LocalValue = dict_of_codes[key][code] 
        if LocalValue == '': 
            listPtValues.append(0)
        else:
             listPtValues.append(LocalValue)
    # print("patient values: ", listPtValues)
    dict_encoded_pt_AL_Dx_sex[ptID] = listPtValues

df_all_encoded_AL_dx_sex = pd.DataFrame.from_dict(dict_encoded_pt_AL_Dx_sex, orient='index', columns=["Amyloid Status", "Dx", "Sex"])

# df_all_encoded_AL_dx_sex.columns = ["Amyloid Status", "Dx", "Sex"]

In [15]:
#getting age and survival 
df_all_ages = pd.DataFrame.from_dict(patientDictAge, orient= 'index', columns= ['Age'])
df_all_survival = pd.DataFrame.from_dict(patientDictSurvival, orient='index', columns= ['Survival (in months)'])

#combining for demographics

df_all_demographics = pd.concat([df_all_encoded_AL_dx_sex, df_all_ages, df_all_survival], axis= 1)

#slicing for amyloid known and unknown

df_ALKnown_demographics = df_all_demographics.loc[listALKnown]
df_ALOther_demographics = df_all_demographics.loc[listOther]

outputToExcel(df_data = df_ALKnown_demographics, fileName_header= "AL-KnownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "demographics")
outputToExcel(df_data = df_ALOther_demographics, fileName_header= "AL-UnknownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "demographics")

### Fits and Descriptive statistics for a section of material

In [16]:
#descriptive statistics of raw lab values 
descriptive_stats_dict = {}

for pt in list(patientDictAmyloid.keys()):
    ptLabDataAll = dictOfFilteredLabs[pt]

    #create structure for one patient 
    labTracker = 0
    # LabDescStatsSeries = pd.Series()
    for lab in columnNames:
        ptLabData = ptLabDataAll[lab]

        #descriptive stats
        LabDescStats = ptLabData.dropna().astype(int).describe()
        orig_indices = LabDescStats.index
        newIndices = [lab + " Desc " + str(idx) for idx in orig_indices]
        LabDescStats.index = newIndices
        
        #check to see if STD is not a number... or if there is only 1 data point 
        labCounts = lab + " Desc " + "count" 
        labSTD = lab + " Desc " + "std" 
        if LabDescStats[labCounts] == 1: 
            LabDescStats[labSTD] = one_point_STD

        #add the data to a constant list

        if labTracker == 0: 
            #if this is a new patient and the first lab
            #create the series variable
            LabDescStatsSeries = LabDescStats.copy()
        if labTracker > 0: 
            #same patient as last patient, new lab
            #add the data to the series
            LabDescStatsSeries = LabDescStatsSeries._append(LabDescStats)

        #increment to track if the data is for a new patient or not
        labTracker = labTracker + 1
    #saving the data from the patient to a dictionary
    descriptive_stats_dict[pt] = LabDescStatsSeries

#converting the dictionary to a dataframe
#rows are patient IDs
#columns are the descriptive statistics for a particular lab
df_all_DescriptiveStats = pd.DataFrame.from_dict(descriptive_stats_dict, orient='index')


In [17]:
#fill missing data
penalty_fillna_descriptiveStats =-1
df_all_DescriptiveStats = df_all_DescriptiveStats.fillna(penalty_fillna_descriptiveStats)

In [18]:

#collecting only those which are AL +/- 
df_ALKnown_DescriptiveStats = df_all_DescriptiveStats.loc[listALKnown]
df_ALOther_DescriptiveStats = df_all_DescriptiveStats.loc[listOther]

outputToExcel(df_data = df_ALKnown_DescriptiveStats, fileName_header= "AL-KnownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "labs-descriptive")
outputToExcel(df_data = df_ALOther_DescriptiveStats, fileName_header= "AL-UnknownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "labs-descriptive")

# fits of the data

### defintion of functions for fitting data

In [19]:
#definitions of functions 
def linear_model(x, a, b):
    return a * x + b 

def linear_model_forceB(x, a):
    return a * x 

def quadratic_model(x, a, b, c):
    return a * x**2 + b*x + c

# Define model systems
def quadratic_model_force(x, a, b):
    return a * x **2 + b*x 


### penalty information

In [20]:
#penalty parameters for data of length 0 
            # regression_parameters = {
            #     # 'patient ID' : ptID_test,
            #     # 'lab' : lab,
            #     # 'model type' : model_type,
            #     # 'number of data points': dataLength,
            #     'coef' : 'unable to calculate',
            #     'intercpet': 'unable to calculate', 
            #     'R2' : 'unable to calculate',
            #     'residual error': 'unable to calculate'
            # }
penalty_coef= -20000
penalty_intercept = -20000
penalty_R2 = -20000
penalty_residual_error = -20000
penalty_fillna = -20000

### linear fit with forced intercept and threshold set to 2 data points

In [21]:
# ptID = "E-01"
LabFitsDict = {}
for ptID in list(patientDictAmyloid.keys()):
    ptLabs = dictOfFilteredLabs[ptID]
    model_type = "Linear Force"

    regression_parameters = {} #dictionary for data 
    labTracker = 0 #tracking if this is the first lab for a patient 
    for lab in columnNames: 
        # print("this is the lab: ", lab)
        model_type = "Linear Forced"
        y_column_name = lab
        df_Labs_results_2 = ptLabs[[x_column_name, y_column_name]]

        #drop rows were there is either Null or 0 data.  This must be done as a dataframe so that the data remains the same length 
        df_Labs_filtered_notNA = df_Labs_results_2.dropna(axis='rows') #drops the row if either time or lab result are missing

        #first remove any y which is negative
        df_Labs_filtered_notNA_2 = df_Labs_filtered_notNA.loc[df_Labs_filtered_notNA[y_column_name]>= 0]

        #get the x values to an array --> this is time
        x_values = df_Labs_filtered_notNA_2[[x_column_name]]

        #get the y values to an array --> these are the lab results
        y_values_raw = df_Labs_filtered_notNA_2[y_column_name]

        #check the length of the data
        dataLength = len(y_values_raw)

        if dataLength > 1: 
            #find the starting value at baseline
            ptBaseline2 = y_values_raw.iloc[0]
            y_values_adjusted = y_values_raw - ptBaseline2
            y_values = y_values_adjusted
            #flattening data so you can work with it in curve fit 
            x_flat = x_values.squeeze()
            y_flat = y_values.squeeze()

            # Fit the model to the adjusted data
            popt, pcov = curve_fit(linear_model_forceB, x_flat, y_flat)

            # Get the slope (m)
            m = popt[0]

            # Create the linear fit for calculating quality of fit
            y_predict = linear_model_forceB(x_values, m) #+ ptBaseline2

            # #predict the y values using the fit model 
            y_predict_ln = (y_predict) + ptBaseline2 # corrects for baseline shift

            #calc R^2
            r_squared = r2_score(y_flat, y_predict)

            #calc residual error (mean squared error)
            residual_error = mean_squared_error(y_flat, y_predict)
            
            residuals = y_values - y_predict.squeeze()
            residualsIndex = x_values.squeeze()
            residuals = residuals.set_axis(residualsIndex)

            #assigning the parameters to a dataframe to place into
            regression_parameters = {
                'coef' : m,
                'intercpet': ptBaseline2, 
                'R2' : r_squared, 
                'residual error': residual_error
            }
            #convert the dictionary to a series so you can reassign the indices 
            regression_params_2series = pd.Series(regression_parameters)

            #get descriptive information about the residuals 
            descResiduals = residuals.describe()

            orig_indices_descResiduals = descResiduals.index
            newIndices_descResiduals = [lab + " " + model_type + " RegresDesc " + str(idx) for idx in orig_indices_descResiduals]
            descResiduals.index = newIndices_descResiduals

            #check to see if STD is not a number... or if there is only 1 data point 
            labCounts =lab + " " + model_type + " RegresDesc " + "count" 
            labSTD = lab + " " + model_type + " RegresDesc " + "std" 
            if descResiduals[labCounts] == 1: 
                descResiduals[labSTD] = 0

            orig_indices_regresparams = regression_params_2series.index
            newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
            regression_params_2series.index = newIndices_regresparams

            #add the data to a constant list

            if labTracker == 0: 
                #if this is a new patient and the first lab
                #create the series variable
                LabFitSeries = regression_params_2series.copy()
                LabFitSeries = LabFitSeries._append(descResiduals)
            if labTracker > 0: 
                #same patient as last patient, new lab
                #add the data to the series
                LabFitSeries = LabFitSeries._append(regression_params_2series)
                LabFitSeries = LabFitSeries._append(descResiduals)
            labTracker = labTracker + 1 

        elif dataLength == 1: 
            #find the starting value at baseline
            ptBaseline2 = y_values_raw.iloc[0]
            y_values_adjusted = y_values_raw - ptBaseline2
            y_values = y_values_adjusted
            #flattening data so you can work with it in curve fit 
            x_flat = x_values.squeeze()
            y_flat = y_values.squeeze()

            # Get the slope (m)
            m = 0

            # Create the linear fit for calculating quality of fit
            y_predict = linear_model_forceB(x_values, m) #+ ptBaseline2

            # #predict the y values using the fit model 
            y_predict_ln = (y_predict) + ptBaseline2 # corrects for baseline shift

            #calc R^2
            r_squared = one_point_r2

            #calc residual error (mean squared error)
            residual_error = one_point_residual_error
            
            residuals = y_values - y_predict.squeeze()

            #assigning the parameters to a dataframe to place into
            regression_parameters = {
                'coef' : m,
                'intercpet': ptBaseline2, 
                'R2' : r_squared, 
                'residual error': residual_error
            }
            #convert the dictionary to a series so you can reassign the indices 
            regression_params_2series = pd.Series(regression_parameters)

            #get descriptive information about the residuals 
            descResiduals = residuals.describe()

            orig_indices_descResiduals = descResiduals.index
            newIndices_descResiduals = [lab + " " + model_type + " RegresDesc " + str(idx) for idx in orig_indices_descResiduals]
            descResiduals.index = newIndices_descResiduals

            #check to see if STD is not a number... or if there is only 1 data point 
            labCounts =lab + " " + model_type + " RegresDesc " + "count" 
            labSTD = lab + " " + model_type + " RegresDesc " + "std" 
            if descResiduals[labCounts] == 1: 
                descResiduals[labSTD] = one_point_STD

            orig_indices_regresparams = regression_params_2series.index
            newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
            regression_params_2series.index = newIndices_regresparams

            #add the data to a constant list

            if labTracker == 0: 
                #if this is a new patient and the first lab
                #create the series variable
                LabFitSeries = regression_params_2series.copy()
                LabFitSeries = LabFitSeries._append(descResiduals)
            if labTracker > 0: 
                #same patient as last patient, new lab
                #add the data to the series
                LabFitSeries = LabFitSeries._append(regression_params_2series)
                LabFitSeries = LabFitSeries._append(descResiduals)
            labTracker = labTracker + 1 
        else:
            #not enough data to perform regression
            regression_parameters = {
                # 'patient ID' : ptID_test,
                # 'lab' : lab,
                # 'model type' : model_type,
                # 'number of data points': dataLength,
                'coef' : penalty_coef,
                'intercpet': penalty_intercept, 
                'R2' : penalty_R2,
                'residual error': penalty_residual_error
            }
            #convert the dictionary to a series so you can reassign the indices 
            regression_params_2series = pd.Series(regression_parameters)
            orig_indices_regresparams = regression_params_2series.index
            newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
            regression_params_2series.index = newIndices_regresparams
            
            if labTracker == 0: 
                #if this is a new patient and the first lab
                #create the series variable
                LabFitSeries = regression_params_2series.copy()
            if labTracker > 0: 
                #same patient as last patient, new lab
                LabFitSeries = LabFitSeries._append(regression_params_2series)
            labTracker = labTracker + 1 

    #saving the data from the patient to a dictionary
    
    LabFitsDict[ptID] = LabFitSeries

#converting the dictionary to a dataframe
#rows are patient IDs
#columns are the descriptive statistics for a particular lab
df_all_LabFitsLinear = pd.DataFrame.from_dict(LabFitsDict, orient='index')

# outputToExcel(df_data = df_all_LabFitsLinear, fileName_header= "AllPts", fileName_Common= "-Summary", fileName_suffix= "_2024-06-28",
#             parent_dir= outputDir, folderName=folderNameOut, sheetName= "Labs-LinFit-Forced")



In [22]:
#fill missing data
df_all_LabFitsLinear = df_all_LabFitsLinear.fillna(penalty_fillna)

In [23]:
# #collecting only those which are AL +/- 
df_ALKnown_LabFitsLinear = df_all_LabFitsLinear.loc[listALKnown]
df_ALOther_LabFitsLinear = df_all_LabFitsLinear.loc[listOther]

outputToExcel(df_data = df_ALKnown_LabFitsLinear, fileName_header= "AL-KnownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "Labs-LinFit-Forced")
outputToExcel(df_data = df_ALOther_LabFitsLinear, fileName_header= "AL-UnknownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "Labs-LinFit-Forced")

### linear fit without forced intercept.  Threshold set to 2 data points.

In [24]:
# ptID = "E-01"
LabFitsDict = {}
for ptID in list(patientDictAmyloid.keys()):
    ptLabs = dictOfFilteredLabs[ptID]
    model_type = "Linear NotForced"

    regression_parameters = {} #dictionary for data 
    labTracker = 0 #tracking if this is the first lab for a patient 
    for lab in columnNames: 
        # print("this is the lab: ", lab)
        # model_type = "Linear Forced"
        y_column_name = lab
        df_Labs_results_2 = ptLabs[[x_column_name, y_column_name]]

        #drop rows were there is either Null or 0 data.  This must be done as a dataframe so that the data remains the same length 
        df_Labs_filtered_notNA = df_Labs_results_2.dropna(axis='rows') #drops the row if either time or lab result are missing

        #first remove any y which is negative
        df_Labs_filtered_notNA_2 = df_Labs_filtered_notNA.loc[df_Labs_filtered_notNA[y_column_name]>= 0]

        #get the x values to an array --> this is time
        x_values = df_Labs_filtered_notNA_2[[x_column_name]]

        #get the y values to an array --> these are the lab results
        y_values_raw = df_Labs_filtered_notNA_2[y_column_name]

        #check the length of the data
        dataLength = len(y_values_raw)

        if dataLength > 1: 
            #find the starting value at baseline
            y_values_adjusted = y_values_raw #- ptBaseline2
            y_values = y_values_adjusted
            #flattening data so you can work with it in curve fit 
            x_flat = x_values.squeeze()
            y_flat = y_values.squeeze()

            #regression
            regression = LinearRegression().fit(x_values, y_values)
            r_squared = regression.score(x_values, y_values)
            regression_coef = regression.coef_
            regression_coef= float(regression_coef)
            regression_intercept = regression.intercept_
            regression_intercept = float(regression_intercept)

            #to find the residual error first predict the y values
            y_predict = regression.predict(x_values)
            y_predict_ln = (y_predict) 
            residual_error = mean_squared_error(y_values, y_predict)
            residuals = y_values - y_predict
            residualsIndex = x_values.squeeze()
            residuals = residuals.set_axis(residualsIndex)

            #assigning the parameters to a dataframe to place into
            regression_parameters = {
                # 'patient ID' : ptID_test,
                # 'lab' : lab,
                # 'model type' : model_type,
                # 'number of data points': dataLength,
                'coef' : regression_coef,
                'intercpet': regression_intercept, 
                'R2' : r_squared, 
                'residual error': residual_error
            }
            #convert the dictionary to a series so you can reassign the indices 
            regression_params_2series = pd.Series(regression_parameters)

            #get descriptive information about the residuals 
            descResiduals = residuals.describe()

            orig_indices_descResiduals = descResiduals.index
            newIndices_descResiduals = [lab + " " + model_type + " RegresDesc " + str(idx) for idx in orig_indices_descResiduals]
            descResiduals.index = newIndices_descResiduals

            orig_indices_regresparams = regression_params_2series.index
            newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
            regression_params_2series.index = newIndices_regresparams

            #check to see if STD is not a number... or if there is only 1 data point 
            labCounts =lab + " " + model_type + " RegresDesc " + "count" 
            labSTD = lab + " " + model_type + " RegresDesc " + "std" 
            if descResiduals[labCounts] == 1: 
                descResiduals[labSTD] = one_point_STD
            
            #add the data to a constant list

            if labTracker == 0: 
                #if this is a new patient and the first lab
                #create the series variable
                LabFitSeries = regression_params_2series.copy()
                LabFitSeries = LabFitSeries._append(descResiduals)
            if labTracker > 0: 
                #same patient as last patient, new lab
                #add the data to the series
                LabFitSeries = LabFitSeries._append(regression_params_2series)
                LabFitSeries = LabFitSeries._append(descResiduals)
            labTracker = labTracker + 1 
        elif dataLength == 1: 
            #find the starting value at baseline
            ptBaseline2 = y_values_raw.iloc[0]
            y_values_adjusted = y_values_raw - ptBaseline2
            y_values = y_values_adjusted
            #flattening data so you can work with it in curve fit 
            x_flat = x_values.squeeze()
            y_flat = y_values.squeeze()

            # Get the slope (m)
            m = 0

            # Create the linear fit for calculating quality of fit
            y_predict = linear_model_forceB(x_values, m) #+ ptBaseline2

            # #predict the y values using the fit model 
            y_predict_ln = (y_predict) + ptBaseline2 # corrects for baseline shift

            #calc R^2
            r_squared = one_point_r2

            #calc residual error (mean squared error)
            residual_error = one_point_residual_error
            
            residuals = y_values - y_predict.squeeze()

            #assigning the parameters to a dataframe to place into
            regression_parameters = {
                'coef' : m,
                'intercpet': ptBaseline2, 
                'R2' : r_squared, 
                'residual error': residual_error
            }
            #convert the dictionary to a series so you can reassign the indices 
            regression_params_2series = pd.Series(regression_parameters)

            #get descriptive information about the residuals 
            descResiduals = residuals.describe()

            orig_indices_descResiduals = descResiduals.index
            newIndices_descResiduals = [lab + " " + model_type + " RegresDesc " + str(idx) for idx in orig_indices_descResiduals]
            descResiduals.index = newIndices_descResiduals

            #check to see if STD is not a number... or if there is only 1 data point 
            labCounts =lab + " " + model_type + " RegresDesc " + "count" 
            labSTD = lab + " " + model_type + " RegresDesc " + "std" 
            if descResiduals[labCounts] == 1: 
                descResiduals[labSTD] = one_point_STD

            orig_indices_regresparams = regression_params_2series.index
            newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
            regression_params_2series.index = newIndices_regresparams

            #add the data to a constant list

            if labTracker == 0: 
                #if this is a new patient and the first lab
                #create the series variable
                LabFitSeries = regression_params_2series.copy()
                LabFitSeries = LabFitSeries._append(descResiduals)
            if labTracker > 0: 
                #same patient as last patient, new lab
                #add the data to the series
                LabFitSeries = LabFitSeries._append(regression_params_2series)
                LabFitSeries = LabFitSeries._append(descResiduals)
            labTracker = labTracker + 1 
        else:
            #not enough data to perform regression
            regression_parameters = {
                # 'patient ID' : ptID_test,
                # 'lab' : lab,
                # 'model type' : model_type,
                # 'number of data points': dataLength,
                'coef' : penalty_coef,
                'intercpet': penalty_intercept, 
                'R2' : penalty_R2,
                'residual error': penalty_residual_error
            }
            #convert the dictionary to a series so you can reassign the indices 
            regression_params_2series = pd.Series(regression_parameters)
            orig_indices_regresparams = regression_params_2series.index
            newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
            regression_params_2series.index = newIndices_regresparams
            if labTracker == 0: 
                #if this is a new patient and the first lab
                #create the series variable
                LabFitSeries = regression_params_2series.copy()
            if labTracker > 0: 
                #same patient as last patient, new lab
                LabFitSeries = LabFitSeries._append(regression_params_2series)
            labTracker = labTracker + 1 

    #saving the data from the patient to a dictionary
    
    LabFitsDict[ptID] = LabFitSeries

#converting the dictionary to a dataframe
#rows are patient IDs
#columns are the descriptive statistics for a particular lab
df_all_LabFitsLinearWB = pd.DataFrame.from_dict(LabFitsDict, orient='index')

# outputToExcel(df_data = df_all_LabFitsLinearWB, fileName_header= "AllPts", fileName_Common= "-Summary", fileName_suffix= "_2024-06-28",
#             parent_dir= outputDir, folderName=folderNameOut, sheetName= "Labs-LinFit")

C:\Users\maega\AppData\Local\Temp\ipykernel_28836\3871560507.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  regression_coef= float(regression_coef)
C:\Users\maega\AppData\Local\Temp\ipykernel_28836\3871560507.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  regression_coef= float(regression_coef)
C:\Users\maega\AppData\Local\Temp\ipykernel_28836\3871560507.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  regression_coef= float(regression_coef)
C:\User

In [25]:
type(regression_intercept)

float

In [26]:
#filling missing data
df_all_LabFitsLinearWB = df_all_LabFitsLinearWB.fillna(penalty_fillna)

In [27]:
#collecting only those which are AL +/- 
df_ALKnown_LabFitsLinearWB = df_all_LabFitsLinearWB.loc[listALKnown]
df_ALOther_LabFitsLinearWB = df_all_LabFitsLinearWB.loc[listOther]

outputToExcel(df_data = df_ALKnown_LabFitsLinearWB, fileName_header= "AL-KnownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "Labs-LinFit")
outputToExcel(df_data = df_ALOther_LabFitsLinearWB, fileName_header= "AL-UnknownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "Labs-LinFit")

### quadratic fits with forced intercept and threshold set to 3 data points

In [28]:
# ptID = "E-01"
LabFitsDict = {}
for ptID in list(patientDictAmyloid.keys()):
    ptLabs = dictOfFilteredLabs[ptID]
    model_type = "Quadratic Force"

    regression_parameters = {} #dictionary for data 
    labTracker = 0 #tracking if this is the first lab for a patient 
    for lab in columnNames: 
        # print("this is the lab: ", lab)
        # model_type = "Linear Forced"
        y_column_name = lab
        df_Labs_results_2 = ptLabs[[x_column_name, y_column_name]]

        #drop rows were there is either Null or 0 data.  This must be done as a dataframe so that the data remains the same length 
        df_Labs_filtered_notNA = df_Labs_results_2.dropna(axis='rows') #drops the row if either time or lab result are missing

        #first remove any y which is negative
        df_Labs_filtered_notNA_2 = df_Labs_filtered_notNA.loc[df_Labs_filtered_notNA[y_column_name]>= 0]

        #get the x values to an array --> this is time
        x_values = df_Labs_filtered_notNA_2[[x_column_name]]

        #get the y values to an array --> these are the lab results
        y_values_raw = df_Labs_filtered_notNA_2[y_column_name]

        #check the length of the data
        dataLength = len(y_values_raw)

        if dataLength > 2: 
            #find the starting value at baseline
            ptBaseline2 = y_values_raw.iloc[0]
            y_values_adjusted = y_values_raw - ptBaseline2
            y_values = y_values_adjusted
            #flattening data so you can work with it in curve fit 
            x_flat = x_values.squeeze()
            y_flat = y_values.squeeze()

            # Fit the model to the adjusted data
            popt, pcov = curve_fit(quadratic_model_force, x_flat, y_flat)

            # Get the slope (m)
            A = popt[0]
            B = popt[1]

            # Create the linear fit for calculating quality of fit
            y_predict = quadratic_model_force(x_values, A, B) #+ ptBaseline2

            # #predict the y values using the fit model 
            y_predict_ln = (y_predict) + ptBaseline2 # corrects for baseline shift

            #calc R^2
            r_squared = r2_score(y_flat, y_predict)

            #calc residual error (mean squared error)
            residual_error = mean_squared_error(y_flat, y_predict)
            
            residuals = y_values - y_predict.squeeze()
            residualsIndex = x_values.squeeze()
            residuals = residuals.set_axis(residualsIndex)

            #assigning the parameters to a dataframe to place into
            regression_parameters = {
                # 'patient ID' : ptID_test,
                # 'lab' : lab,
                # 'model type' : model_type,
                # 'number of data points': dataLength,
                'coef A' : A,
                'coef B': B,
                'intercpet': ptBaseline2, 
                'R2' : r_squared, 
                'residual error': residual_error
            }
            #convert the dictionary to a series so you can reassign the indices 
            regression_params_2series = pd.Series(regression_parameters)

            #get descriptive information about the residuals 
            descResiduals = residuals.describe()

            orig_indices_descResiduals = descResiduals.index
            newIndices_descResiduals = [lab + " " + model_type + " RegresDesc " + str(idx) for idx in orig_indices_descResiduals]
            descResiduals.index = newIndices_descResiduals

            orig_indices_regresparams = regression_params_2series.index
            newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
            regression_params_2series.index = newIndices_regresparams
            
            #check to see if STD is not a number... or if there is only 1 data point 
            labCounts =lab + " " + model_type + " RegresDesc " + "count" 
            labSTD = lab + " " + model_type + " RegresDesc " + "std" 
            if descResiduals[labCounts] == 1: 
                descResiduals[labSTD] = one_point_STD

            #add the data to a constant list

            if labTracker == 0: 
                #if this is a new patient and the first lab
                #create the series variable
                LabFitSeries = regression_params_2series.copy()
                LabFitSeries = LabFitSeries._append(descResiduals)
            if labTracker > 0: 
                #same patient as last patient, new lab
                #add the data to the series
                LabFitSeries = LabFitSeries._append(regression_params_2series)
                LabFitSeries = LabFitSeries._append(descResiduals)
            labTracker = labTracker + 1 

        elif dataLength == 2: 
            #find the starting value at baseline
            ptBaseline2 = y_values_raw.iloc[0]
            y_values_adjusted = y_values_raw - ptBaseline2
            y_values = y_values_adjusted
            #flattening data so you can work with it in curve fit 
            x_flat = x_values.squeeze()
            y_flat = y_values.squeeze()

            # Fit the model to the adjusted data
            popt, pcov = curve_fit(linear_model_forceB, x_flat, y_flat)

            # Get the slope (m)
            m = popt[0]

            # Create the linear fit for calculating quality of fit
            y_predict = linear_model_forceB(x_values, m) #+ ptBaseline2

            # #predict the y values using the fit model 
            y_predict_ln = (y_predict) + ptBaseline2 # corrects for baseline shift

            #calc R^2
            r_squared = r2_score(y_flat, y_predict)

            #calc residual error (mean squared error)
            residual_error = mean_squared_error(y_flat, y_predict)
            
            residuals = y_values - y_predict.squeeze()
            residualsIndex = x_values.squeeze()
            residuals = residuals.set_axis(residualsIndex)

            #assigning the parameters to a dataframe to place into
            regression_parameters = {
                # 'patient ID' : ptID_test,
                # 'lab' : lab,
                # 'model type' : model_type,
                # 'number of data points': dataLength,
                'coef A' : 0,
                'coef B': m,
                'intercpet': ptBaseline2, 
                'R2' : r_squared, 
                'residual error': residual_error
            }
            #convert the dictionary to a series so you can reassign the indices 
            regression_params_2series = pd.Series(regression_parameters)

            #get descriptive information about the residuals 
            descResiduals = residuals.describe()

            orig_indices_descResiduals = descResiduals.index
            newIndices_descResiduals = [lab + " " + model_type + " RegresDesc " + str(idx) for idx in orig_indices_descResiduals]
            descResiduals.index = newIndices_descResiduals

            #check to see if STD is not a number... or if there is only 1 data point 
            labCounts =lab + " " + model_type + " RegresDesc " + "count" 
            labSTD = lab + " " + model_type + " RegresDesc " + "std" 
            if descResiduals[labCounts] == 1: 
                descResiduals[labSTD] = one_point_STD

            orig_indices_regresparams = regression_params_2series.index
            newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
            regression_params_2series.index = newIndices_regresparams

            #add the data to a constant list

            if labTracker == 0: 
                #if this is a new patient and the first lab
                #create the series variable
                LabFitSeries = regression_params_2series.copy()
                LabFitSeries = LabFitSeries._append(descResiduals)
            if labTracker > 0: 
                #same patient as last patient, new lab
                #add the data to the series
                LabFitSeries = LabFitSeries._append(regression_params_2series)
                LabFitSeries = LabFitSeries._append(descResiduals)
            labTracker = labTracker + 1 

        elif dataLength == 1: 
            #find the starting value at baseline
            ptBaseline2 = y_values_raw.iloc[0]
            y_values_adjusted = y_values_raw - ptBaseline2
            y_values = y_values_adjusted
            #flattening data so you can work with it in curve fit 
            x_flat = x_values.squeeze()
            y_flat = y_values.squeeze()

            #coefs 
            A = 0
            B = 0

            # Create the linear fit for calculating quality of fit
            y_predict = quadratic_model_force(x_values, A, B) #+ ptBaseline2

            # #predict the y values using the fit model 
            y_predict_ln = (y_predict) + ptBaseline2 # corrects for baseline shift

            #calc R^2
            r_squared = one_point_r2

            #calc residual error (mean squared error)
            residual_error = one_point_residual_error
            
            residuals = y_values - y_predict.squeeze()

            #assigning the parameters to a dataframe to place into
            regression_parameters = {
                # 'number of data points': dataLength,
                'coef A' : A,
                'coef B': B,
                'intercpet': ptBaseline2, 
                'R2' : r_squared, 
                'residual error': residual_error
            }
            #convert the dictionary to a series so you can reassign the indices 
            regression_params_2series = pd.Series(regression_parameters)

            #get descriptive information about the residuals 
            descResiduals = residuals.describe()

            orig_indices_descResiduals = descResiduals.index
            newIndices_descResiduals = [lab + " " + model_type + " RegresDesc " + str(idx) for idx in orig_indices_descResiduals]
            descResiduals.index = newIndices_descResiduals

            #check to see if STD is not a number... or if there is only 1 data point 
            labCounts =lab + " " + model_type + " RegresDesc " + "count" 
            labSTD = lab + " " + model_type + " RegresDesc " + "std" 
            if descResiduals[labCounts] == 1: 
                descResiduals[labSTD] = one_point_STD

            orig_indices_regresparams = regression_params_2series.index
            newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
            regression_params_2series.index = newIndices_regresparams

            #add the data to a constant list

            if labTracker == 0: 
                #if this is a new patient and the first lab
                #create the series variable
                LabFitSeries = regression_params_2series.copy()
                LabFitSeries = LabFitSeries._append(descResiduals)
            if labTracker > 0: 
                #same patient as last patient, new lab
                #add the data to the series
                LabFitSeries = LabFitSeries._append(regression_params_2series)
                LabFitSeries = LabFitSeries._append(descResiduals)
            labTracker = labTracker + 1 
        
        else:
            #not enough data to perform regression
            regression_parameters = {
                # 'patient ID' : ptID_test,
                # 'lab' : lab,
                # 'model type' : model_type,
                # 'number of data points': dataLength,
                'coef A' : 0,
                'coef B': penalty_coef,
                'intercpet': penalty_R2, 
                'R2' : penalty_R2,
                'residual error': penalty_residual_error
            }
            #convert the dictionary to a series so you can reassign the indices 
            regression_params_2series = pd.Series(regression_parameters)
            orig_indices_regresparams = regression_params_2series.index
            newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
            regression_params_2series.index = newIndices_regresparams
            if labTracker == 0: 
                #if this is a new patient and the first lab
                #create the series variable
                LabFitSeries = regression_params_2series.copy()
            if labTracker > 0: 
                #same patient as last patient, new lab
                LabFitSeries = LabFitSeries._append(regression_params_2series)
            labTracker = labTracker + 1 

    #saving the data from the patient to a dictionary
    
    LabFitsDict[ptID] = LabFitSeries

#converting the dictionary to a dataframe
#rows are patient IDs
#columns are the descriptive statistics for a particular lab
df_all_LabFitsQuadForce = pd.DataFrame.from_dict(LabFitsDict, orient='index')

# outputToExcel(df_data = df_all_LabFitsQuadForce, fileName_header= "AllPts", fileName_Common= "-Summary", fileName_suffix= "_2024-06-28",
#             parent_dir= outputDir, folderName=folderNameOut, sheetName= "Labs-QuadFit")

In [29]:
#filling missing data
df_all_LabFitsQuadForce = df_all_LabFitsQuadForce.fillna(penalty_fillna)

In [30]:
#collecting only those which are AL +/- 
df_ALKnown_LabFitsQuadForce = df_all_LabFitsQuadForce.loc[listALKnown]
df_ALOther_LabFitsQuadForce = df_all_LabFitsQuadForce.loc[listOther]

outputToExcel(df_data = df_ALKnown_LabFitsQuadForce, fileName_header= "AL-KnownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "Labs-QuadFit")
outputToExcel(df_data = df_ALOther_LabFitsQuadForce, fileName_header= "AL-UnknownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "Labs-QuadFit")

### exponential fits threshold set to 2

In [31]:
# ptID = "E-01"
LabFitsDict = {}
for ptID in list(patientDictAmyloid.keys()):
    ptLabs = dictOfFilteredLabs[ptID]
    model_type = "Exp2 Force"

    regression_parameters = {} #dictionary for data 
    labTracker = 0 #tracking if this is the first lab for a patient 
    for lab in columnNames: 
        # print("this is the lab: ", lab)
        # model_type = "Linear Forced"
        y_column_name = lab
        df_Labs_results_2 = ptLabs[[x_column_name, y_column_name]]

        #drop rows were there is either Null or 0 data.  This must be done as a dataframe so that the data remains the same length 
        df_Labs_filtered_notNA = df_Labs_results_2.dropna(axis='rows') #drops the row if either time or lab result are missing

        #first remove any y which is negative
        df_Labs_filtered_notNA_2 = df_Labs_filtered_notNA.loc[df_Labs_filtered_notNA[y_column_name]> 0]

        #get the x values to an array --> this is time
        x_values = df_Labs_filtered_notNA_2[[x_column_name]]

        #get the y values to an array --> these are the lab results
        y_values_raw = df_Labs_filtered_notNA_2[y_column_name]

        #check the length of the data
        dataLength = len(y_values_raw)

        if dataLength > 1: 
            #find the starting value at baseline
            y_values_transformed = np.log(y_values_raw)
            ptBaseline2 = y_values_transformed.iloc[0]
            y_values_adjusted = y_values_transformed - ptBaseline2
            y_values = y_values_adjusted
            #flattening data so you can work with it in curve fit 
            x_flat = x_values.squeeze()
            y_flat = y_values.squeeze()

            # Fit the model to the adjusted data
            popt, pcov = curve_fit(linear_model_forceB, x_flat, y_flat)

            # get properties
            m = popt[0]

            # Create the linear fit for calculating quality of fit
            #these values are still ln transformed and shifted, compare to y_values
            y_predict = linear_model_forceB(x_values, m) #+ ptBaseline2

            # adjust the predicted values by adding the baseline and 
            # converting back to the original units
            y_predict_ln = np.exp((y_predict) + ptBaseline2) 

            #calc R^2
            r_squared = r2_score(y_flat, y_predict)

            #calc residual error (mean squared error)
            residual_error = mean_squared_error(y_flat, y_predict)
            
            residuals = y_values - y_predict.squeeze()
            residualsIndex = x_values.squeeze()
            residuals = residuals.set_axis(residualsIndex)

            #assigning the parameters to a dataframe to place into
            regression_parameters = {
                # 'patient ID' : ptID_test,
                # 'lab' : lab,
                # 'model type' : model_type,
                # 'number of data points': dataLength,
                'coef' : m,
                'intercpet': ptBaseline2, 
                'R2' : r_squared, 
                'residual error': residual_error
            }
            #convert the dictionary to a series so you can reassign the indices 
            regression_params_2series = pd.Series(regression_parameters)

            #get descriptive information about the residuals 
            descResiduals = residuals.describe()

            orig_indices_descResiduals = descResiduals.index
            newIndices_descResiduals = [lab + " " + model_type + " RegresDesc " + str(idx) for idx in orig_indices_descResiduals]
            descResiduals.index = newIndices_descResiduals

            orig_indices_regresparams = regression_params_2series.index
            newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
            regression_params_2series.index = newIndices_regresparams
            
            #check to see if STD is not a number... or if there is only 1 data point 
            labCounts =lab + " " + model_type + " RegresDesc " + "count" 
            labSTD = lab + " " + model_type + " RegresDesc " + "std" 
            if descResiduals[labCounts] == 1: 
                descResiduals[labSTD] = one_point_STD

            #add the data to a constant list

            if labTracker == 0: 
                #if this is a new patient and the first lab
                #create the series variable
                LabFitSeries = regression_params_2series.copy()
                LabFitSeries = LabFitSeries._append(descResiduals)
            if labTracker > 0: 
                #same patient as last patient, new lab
                #add the data to the series
                LabFitSeries = LabFitSeries._append(regression_params_2series)
                LabFitSeries = LabFitSeries._append(descResiduals)
            labTracker = labTracker + 1 

        elif dataLength == 1: 
            #find the starting value at baseline
            ptBaseline2 = y_values_raw.iloc[0]
            y_values_adjusted = y_values_raw - ptBaseline2
            y_values = y_values_adjusted
            #flattening data so you can work with it in curve fit 
            x_flat = x_values.squeeze()
            y_flat = y_values.squeeze()

            # Get the slope (m)
            m = 0

            # Create the linear fit for calculating quality of fit
            y_predict = linear_model_forceB(x_values, m) #+ ptBaseline2

            # #predict the y values using the fit model 
            y_predict_ln = (y_predict) + ptBaseline2 # corrects for baseline shift

            #calc R^2
            r_squared = one_point_r2

            #calc residual error (mean squared error)
            residual_error = one_point_residual_error
            
            residuals = y_values - y_predict.squeeze()

            #assigning the parameters to a dataframe to place into
            regression_parameters = {
                'coef' : m,
                'intercpet': ptBaseline2, 
                'R2' : r_squared, 
                'residual error': residual_error
            }
            #convert the dictionary to a series so you can reassign the indices 
            regression_params_2series = pd.Series(regression_parameters)

            #get descriptive information about the residuals 
            descResiduals = residuals.describe()

            orig_indices_descResiduals = descResiduals.index
            newIndices_descResiduals = [lab + " " + model_type + " RegresDesc " + str(idx) for idx in orig_indices_descResiduals]
            descResiduals.index = newIndices_descResiduals

            #check to see if STD is not a number... or if there is only 1 data point 
            labCounts =lab + " " + model_type + " RegresDesc " + "count" 
            labSTD = lab + " " + model_type + " RegresDesc " + "std" 
            if descResiduals[labCounts] == 1: 
                descResiduals[labSTD] = one_point_STD

            orig_indices_regresparams = regression_params_2series.index
            newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
            regression_params_2series.index = newIndices_regresparams

            #add the data to a constant list

            if labTracker == 0: 
                #if this is a new patient and the first lab
                #create the series variable
                LabFitSeries = regression_params_2series.copy()
                LabFitSeries = LabFitSeries._append(descResiduals)
            if labTracker > 0: 
                #same patient as last patient, new lab
                #add the data to the series
                LabFitSeries = LabFitSeries._append(regression_params_2series)
                LabFitSeries = LabFitSeries._append(descResiduals)
            labTracker = labTracker + 1 
        else:
            #not enough data to perform regression
            regression_parameters = {
                # 'patient ID' : ptID_test,
                # 'lab' : lab,
                # 'model type' : model_type,
                # 'number of data points': dataLength,
                'coef' : penalty_coef,
                'intercpet': penalty_intercept, 
                'R2' : penalty_R2,
                'residual error': penalty_residual_error
            }
            #convert the dictionary to a series so you can reassign the indices 
            regression_params_2series = pd.Series(regression_parameters) 
            orig_indices_regresparams = regression_params_2series.index
            newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
            regression_params_2series.index = newIndices_regresparams
            if labTracker == 0: 
                #if this is a new patient and the first lab
                #create the series variable
                LabFitSeries = regression_params_2series.copy()
            if labTracker > 0: 
                #same patient as last patient, new lab
                LabFitSeries = LabFitSeries._append(regression_params_2series)
            labTracker = labTracker + 1 

    #saving the data from the patient to a dictionary
    
    LabFitsDict[ptID] = LabFitSeries

#converting the dictionary to a dataframe
#rows are patient IDs
#columns are the descriptive statistics for a particular lab
df_all_LabFitsExp2Force = pd.DataFrame.from_dict(LabFitsDict, orient='index')

# outputToExcel(df_data = df_all_LabFitsExp2Force, fileName_header= "AllPts", fileName_Common= "-Summary", fileName_suffix= "_2024-06-28",
#             parent_dir= outputDir, folderName=folderNameOut, sheetName= "Labs-Exp2Forced")

In [32]:
#filling missing data
df_all_LabFitsExp2Force = df_all_LabFitsExp2Force.fillna(penalty_fillna)

In [33]:
#collecting only those which are AL +/- 
df_ALKnown_LabFitsExp2Force = df_all_LabFitsExp2Force.loc[listALKnown]
df_ALOther_LabFitsExp2Force = df_all_LabFitsExp2Force.loc[listOther]

outputToExcel(df_data = df_ALKnown_LabFitsExp2Force, fileName_header= "AL-KnownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "Labs-Exp2Forced")
outputToExcel(df_data = df_ALOther_LabFitsExp2Force, fileName_header= "AL-UnknownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "Labs-Exp2Forced")

### exponential fit with the threshold set to 3 data points

In [34]:
# # ptID = "E-01"
# LabFitsDict = {}
# for ptID in list(patientDictAmyloid.keys()):
#     ptLabs = dictOfFilteredLabs[ptID]
#     model_type = "Exp3 Force"

#     regression_parameters = {} #dictionary for data 
#     labTracker = 0 #tracking if this is the first lab for a patient 
#     for lab in columnNames: 
#         # print("this is the lab: ", lab)
#         # model_type = "Linear Forced"
#         y_column_name = lab
#         df_Labs_results_2 = ptLabs[[x_column_name, y_column_name]]

#         #drop rows were there is either Null or 0 data.  This must be done as a dataframe so that the data remains the same length 
#         df_Labs_filtered_notNA = df_Labs_results_2.dropna(axis='rows') #drops the row if either time or lab result are missing

#         #first remove any y which is negative
#         df_Labs_filtered_notNA_2 = df_Labs_filtered_notNA.loc[df_Labs_filtered_notNA[y_column_name]> 0]

#         #get the x values to an array --> this is time
#         x_values = df_Labs_filtered_notNA_2[[x_column_name]]

#         #get the y values to an array --> these are the lab results
#         y_values_raw = df_Labs_filtered_notNA_2[y_column_name]

#         #check the length of the data
#         dataLength = len(y_values_raw)

#         if dataLength > 2: 
#             #find the starting value at baseline
#             y_values_transformed = np.log(y_values_raw)
#             ptBaseline2 = y_values_transformed.iloc[0]
#             y_values_adjusted = y_values_transformed - ptBaseline2
#             y_values = y_values_adjusted
#             #flattening data so you can work with it in curve fit 
#             x_flat = x_values.squeeze()
#             y_flat = y_values.squeeze()

#             # Fit the model to the adjusted data
#             popt, pcov = curve_fit(linear_model_forceB, x_flat, y_flat)

#             # get properties
#             m = popt[0]

#             # Create the linear fit for calculating quality of fit
#             #these values are still ln transformed and shifted, compare to y_values
#             y_predict = linear_model_forceB(x_values, m) #+ ptBaseline2

#             # adjust the predicted values by adding the baseline and 
#             # converting back to the original units
#             y_predict_ln = np.exp((y_predict) + ptBaseline2) 

#             #calc R^2
#             r_squared = r2_score(y_flat, y_predict)

#             #calc residual error (mean squared error)
#             residual_error = mean_squared_error(y_flat, y_predict)
            
#             residuals = y_values - y_predict.squeeze()
#             residualsIndex = x_values.squeeze()
#             residuals = residuals.set_axis(residualsIndex)

#             #assigning the parameters to a dataframe to place into
#             regression_parameters = {
#                 # 'patient ID' : ptID_test,
#                 # 'lab' : lab,
#                 # 'model type' : model_type,
#                 # 'number of data points': dataLength,
#                 'coef' : m,
#                 'intercpet': ptBaseline2, 
#                 'R2' : r_squared, 
#                 'residual error': residual_error
#             }
#             #convert the dictionary to a series so you can reassign the indices 
#             regression_params_2series = pd.Series(regression_parameters)

#             #get descriptive information about the residuals 
#             descResiduals = residuals.describe()

#             orig_indices_descResiduals = descResiduals.index
#             newIndices_descResiduals = [lab + " " + model_type + " RegresDesc " + str(idx) for idx in orig_indices_descResiduals]
#             descResiduals.index = newIndices_descResiduals

#             orig_indices_regresparams = regression_params_2series.index
#             newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
#             regression_params_2series.index = newIndices_regresparams
            
#             #add the data to a constant list

#             if labTracker == 0: 
#                 #if this is a new patient and the first lab
#                 #create the series variable
#                 LabFitSeries = regression_params_2series.copy()
#                 LabFitSeries = LabFitSeries._append(descResiduals)
#             if labTracker > 0: 
#                 #same patient as last patient, new lab
#                 #add the data to the series
#                 LabFitSeries = LabFitSeries._append(regression_params_2series)
#                 LabFitSeries = LabFitSeries._append(descResiduals)
#             labTracker = labTracker + 1 
#         else:
#             #not enough data to perform regression
#             regression_parameters = {
#                 # 'patient ID' : ptID_test,
#                 # 'lab' : lab,
#                 # 'model type' : model_type,
#                 # 'number of data points': dataLength,
#                 'coef' : 'unable to calculate',
#                 'intercpet': 'unable to calculate', 
#                 'R2' : 'unable to calculate',
#                 'residual error': 'unable to calculate'
#             }
#             #convert the dictionary to a series so you can reassign the indices 
#             regression_params_2series = pd.Series(regression_parameters)
#             orig_indices_regresparams = regression_params_2series.index
#             newIndices_regresparams = [lab + " " + model_type + " " + str(idx) for idx in orig_indices_regresparams]
#             regression_params_2series.index = newIndices_regresparams
#             if labTracker == 0: 
#                 #if this is a new patient and the first lab
#                 #create the series variable
#                 LabFitSeries = regression_params_2series.copy()
#             if labTracker > 0: 
#                 #same patient as last patient, new lab
#                 LabFitSeries = LabFitSeries._append(regression_params_2series)
#             labTracker = labTracker + 1 

#     #saving the data from the patient to a dictionary
    
#     LabFitsDict[ptID] = LabFitSeries

# #converting the dictionary to a dataframe
# #rows are patient IDs
# #columns are the descriptive statistics for a particular lab
# df_all_LabFitsExp3Force = pd.DataFrame.from_dict(LabFitsDict, orient='index')

# outputToExcel(df_data = df_all_LabFitsExp3Force, fileName_header= "AllPts", fileName_Common= "-Summary", fileName_suffix= "_2024-06-28",
#             parent_dir= outputDir, folderName=folderNameOut, sheetName= "Labs-Exp3Forced")

### getting the echo data for all patients

In [35]:
penalty_echo = -1


In [36]:
#list of columns of interest
echoColumnsOfInterest = ["1st_EF(avg%)", "1st_Diastolic grade", "1st_BSA", "1st_PWT(cm)",
                         "1st_IVS(cm)", "1st_lvid(cm)", "1st_LA_dimension (cm)","1st_Lvmass",
                         "1st_LVMI","1st_RWT", "1st_LA_grade", "1st_mv_grade", "1st_AV_grade",
                         "1st_rvsp(mmHg)", "1st_LVH", "1stGLS", "1stGLSreported", "use1stGLS",
                         "1st_outerLS", "1st_midLS", "1st_innerLS", "1st_echoOSH"]

dictOfEchoDataAll = {}

EchoTimeColumn ='time from diagnosis to first echo (months)'

PatientCounter = 0
for pt in (patientDictEcho.keys()):
    #to collect and filter the patient labs into a dictionary
    df_Echo = patientDictEcho[pt]

    orig_columns = df_Echo.columns

    #filter the patient labs
    columnNames = FilterList(orig_columns, keyWords_primary = echoColumnsOfInterest)
    
    testNames = [EchoTimeColumn]
    testNames.extend(columnNames) #addition of time label


    ptEcho_results= df_Echo[testNames] #new data frame of just results

    #replacing string with a penalty value
    ptEcho_results = ptEcho_results.applymap(lambda x: penalty_echo if isinstance(x, str) else x)

    current_indices = ptEcho_results.index
    new_indices = ['Echo ' + str(idx) for idx in current_indices]
    ptEcho_results.index = new_indices

    #add the appropriate index
    ptEcho_results.insert(0, "DeID", [pt])
    ptEcho_results = ptEcho_results.set_index('DeID')
    ptEcho_results = ptEcho_results.fillna(value = -1)

    dictOfEchoDataAll[pt] = ptEcho_results

    if PatientCounter == 0: 
        #create dataframe
        df_all_EchoTest = ptEcho_results.copy()
    if PatientCounter > 0: 
        df_all_EchoTest = df_all_EchoTest._append(ptEcho_results)
    PatientCounter = PatientCounter + 1

# outputToExcel(df_data = df_all_EchoTest, fileName_header= "AllPts", fileName_Common= "-Summary", fileName_suffix= "_2024-06-28",
#         parent_dir= outputDir, folderName=folderNameOut, sheetName= "Echos")

C:\Users\maega\AppData\Local\Temp\ipykernel_28836\2646366814.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ptEcho_results = ptEcho_results.applymap(lambda x: penalty_echo if isinstance(x, str) else x)
C:\Users\maega\AppData\Local\Temp\ipykernel_28836\2646366814.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ptEcho_results = ptEcho_results.applymap(lambda x: penalty_echo if isinstance(x, str) else x)
C:\Users\maega\AppData\Local\Temp\ipykernel_28836\2646366814.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ptEcho_results = ptEcho_results.applymap(lambda x: penalty_echo if isinstance(x, str) else x)
C:\Users\maega\AppData\Local\Temp\ipykernel_28836\2646366814.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ptEcho_results = ptEcho_results.applymap(lambda x: penalty_echo if isinstance(x, str) else x)
C:\Users\maega\A

In [37]:
#collecting only those which are AL +/- 
df_ALKnown_Echo = df_all_EchoTest.loc[listALKnown]
df_ALOther_Echo = df_all_EchoTest.loc[listOther]

outputToExcel(df_data = df_ALKnown_Echo, fileName_header= "AL-KnownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "Echos")
outputToExcel(df_data = df_ALOther_Echo, fileName_header= "AL-UnknownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "Echos")

### treatement information

In [38]:
#collecting information about treatments patient has recieved

#each condition appends 2 numbers
#first number is a boolean if the patient has data
#the second number is the total number of treatments the patient has had.

#list of dictionaries 
pt_Tx_dict_to_iterate = [patientDictChemo, patientDictImmuno, patientDictHeme, patientDictRad, patientDictOther]

dictPt_Tx_Summary = {}

for ptID in list(patientDictAge.keys()): 
    #get the treatment information for that patient
    ptTxList = [] 
    for tx in pt_Tx_dict_to_iterate:
        if type(tx[ptID]) != bool: 
            ptTxList.append(1)
            #get the length of the data
            numb_txs = len(tx[ptID])
            ptTxList.append(numb_txs)
        else:
            ptTxList.append(0)
            ptTxList.append(0)

    dictPt_Tx_Summary[ptID] = ptTxList

df_all_tx_summary = pd.DataFrame.from_dict(dictPt_Tx_Summary, orient='index')
df_all_tx_summary.columns = ["ChemoTx", "ChemoTx Number", "ImmunoTx", "ImmunoTx Number", "HemeTx", "HemeTx Number", 
"RadTx", "RadTx Number", "OtherTx", "OtherTx Number"]

In [39]:
#splitting dataframe by amyloid status
df_AL_known_tx_summary = df_all_tx_summary.loc[listALKnown]
df_AL_Unknown_tx_summary = df_all_tx_summary.loc[listOther]

In [40]:
outputToExcel(df_data = df_AL_known_tx_summary, fileName_header= "AL-KnownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "txSummary")
outputToExcel(df_data = df_AL_Unknown_tx_summary, fileName_header= "AL-UnknownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "txSummary")

### physician note information

In [41]:
#dictionaries for conversion
dictECOGtoKPS = {0 : 95,
                 1 : 75, 
                 2 : 55, 
                 3 : 35, 
                 4 : 15, 
                 5 : 0}

dictKPStoECOG = {100: 0, 90: 0, 
                 80: 1, 70: 1, 
                 60: 2, 50: 2, 
                 40: 3 , 30: 3, 
                 20: 4, 10: 4, 0: 5 }

dictClinicalImpression = {
    "Complete Response/No Evidence of Disease": 4,
    "Stable Disease": 1,
    "Partial Response": 3, 
    "Progressive Disease": 2, 
    "Not Applicable": 0,
    "Not Assessable": 0}
#this preserves NAN values 

#create new dictionary for storing the scored notes information
dictMDnotesScored = {}
listPtsMissingNotes = []

#call each patient in the dictionary storing patient note information
for ptID in list(patientDictNotes.keys()):
    #check to make sure the patient has note data
    if not isinstance(patientDictNotes[ptID], bool):
        #get the patient notes
        ptNotes = patientDictNotes[ptID]

        #column names of interest
        listColumnNames = ['time from diagnosis to note (months)', 'ECOG Performance Status', 'Karnofsky Performance Status', 'Clinical Impression of Disease']
        ptNotesCorrected = ptNotes[listColumnNames]
        
        #convert the Karnofsky scores to integers
        ptNotesCorrected['Karnofsky Performance Status'] = ptNotesCorrected['Karnofsky Performance Status'].astype(str).str.extract('(\d+)').astype(float).astype('Int64')


        for i in range(len(ptNotesCorrected['time from diagnosis to note (months)'])):
            #get the scores for that row
            ptTime = ptNotesCorrected.at[i, 'time from diagnosis to note (months)']
            ptECOG = ptNotesCorrected.at[i,'ECOG Performance Status']
            ptKPS = ptNotesCorrected.at[i, 'Karnofsky Performance Status']
            ptClinicalImpression = ptNotesCorrected.at[i, 'Clinical Impression of Disease']

            #if the value of ptECOG was "OTHER", then the row should be dropped.
            if ptECOG == "OTHER":
                # ptNotesCorrected.drop(index = i, inplace=True ) #drops the full row, would need to add time back if interested in clinical impression
                ptECOG = -1 #replace with penalty value
                ptKPS = -1 #replaces with penalty value

            #infer ECOG from Karnofsky where you now have half points
            elif np.isnan(ptECOG) ==True and pd.isna(ptKPS) == False: 
                ptECOG = dictKPStoECOG[ptKPS]

            #infer Karnofsky from ECOG 
            elif pd.isna(ptKPS) == True and np.isnan(ptECOG) ==False: 
                ptKPS = dictECOGtoKPS[ptECOG]

            #save these values back to the dataframe
            if ptECOG != "OTHER":
                ptNotesCorrected.at[i,'ECOG Performance Status'] = ptECOG
                ptNotesCorrected.at[i, 'Karnofsky Performance Status'] = ptKPS

            #checking to see if the patient has a clinical impression input
            #if there is a string, convert to a numeric value using the dictionary
            if type(ptClinicalImpression) == str:
                ptClinicalImpressionNumeric = dictClinicalImpression[ptClinicalImpression]
                ptNotesCorrected.at[i, 'Clinical Impression of Disease'] = ptClinicalImpressionNumeric
                ptNotesCorrected.at[i, 'time from diagnosis to note (months)'] =ptTime
                #you can nest this under the =! "Other" if you find the rows with no ECOG/Karnofsky data to not be helpful
    elif isinstance(patientDictNotes[ptID], bool):
        ptNotesCorrected = False
        listPtsMissingNotes.append([ptID])
                            
    #saving the notes to the new dictionary
    dictMDnotesScored[ptID] = ptNotesCorrected


C:\Users\maega\AppData\Local\Temp\ipykernel_28836\1057607271.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptNotesCorrected['Karnofsky Performance Status'] = ptNotesCorrected['Karnofsky Performance Status'].astype(str).str.extract('(\d+)').astype(float).astype('Int64')
C:\Users\maega\AppData\Local\Temp\ipykernel_28836\1057607271.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptNotesCorrected['Karnofsky Performance Status'] = ptNotesCorrected['Karnofsky Performance Status'].astype(str).str.extra

In [42]:
#now to make this a set of descriptive statistics that can be used in the dataframe
#challenged by nan values
descriptive_stats_dict_notes = {}

for pt in list(dictMDnotesScored.keys()):
    if not isinstance(patientDictNotes[pt], bool):
        ptNoteData = dictMDnotesScored[pt]

        #create structure for one patient 
        columnTracker = 0
        # LabDescStatsSeries = pd.Series()
        listColumnNames_woTime = ['ECOG Performance Status', 'Karnofsky Performance Status', 'Clinical Impression of Disease']

        for col in listColumnNames_woTime:
            ptColData = ptNoteData[col]
            # ptLabData = ptLabData.dropna()

            #descriptive stats
            ColDescStats = ptColData.dropna().astype(int).describe()
            orig_indices = ColDescStats.index
            newIndices = [col + " Desc " + str(idx) for idx in orig_indices]
            ColDescStats.index = newIndices
            
            #check to see if STD is not a number... or if there is only 1 data point 
            colCounts = col + " Desc " + "count" 
            colSTD = col + " Desc " + "std" 
            if ColDescStats[colCounts] == 1: 
                ColDescStats[colSTD] = one_point_STD

            #add the data to a constant list

            if columnTracker == 0: 
                #if this is a new patient and the first lab
                #create the series variable
                ColDescStatsSeries = ColDescStats.copy()
            if columnTracker > 0: 
                #same patient as last patient, new lab
                #add the data to the series
                ColDescStatsSeries = ColDescStatsSeries._append(ColDescStats)

            #increment to track if the data is for a new patient or not
            columnTracker = columnTracker + 1
        #saving the data from the patient to a dictionary
        descriptive_stats_dict_notes[pt] = ColDescStatsSeries

#converting the dictionary to a dataframe
#rows are patient IDs
#columns are the descriptive statistics for a particular lab
df_all_DescriptiveStats_notes = pd.DataFrame.from_dict(descriptive_stats_dict_notes, orient='index')

#fill in missing values and patients
penaltyNoteData = [[-1] * len(df_all_DescriptiveStats_notes.columns)]
columns = list(df_all_DescriptiveStats_notes.columns)
for ptID in listPtsMissingNotes:
    df_penalty_noteData = pd.DataFrame(data=penaltyNoteData, columns= columns, index = ptID)
    df_all_DescriptiveStats_notes = df_all_DescriptiveStats_notes._append(df_penalty_noteData)
#fill missing
df_all_DescriptiveStats_notes = df_all_DescriptiveStats_notes.fillna(-1)

In [43]:
# if "T-01" in listALKnown:
#     listALKnown.remove("T-01")
# if "T-01" in listOther:
#     listOther.remove("T-01")

df_ALKnown_MDNotesDescriptive = df_all_DescriptiveStats_notes.loc[listALKnown]
df_ALOther_MDNotesDescriptive = df_all_DescriptiveStats_notes.loc[listOther]


In [44]:
outputToExcel(df_data = df_ALKnown_MDNotesDescriptive, fileName_header= "AL-KnownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "MDNotes-desc")
outputToExcel(df_data = df_ALOther_MDNotesDescriptive, fileName_header= "AL-UnknownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "MDNotes-desc")

link all the dataframes together

In [47]:
#for amyloid status known patients
df_ALKnown_summary = pd.concat([df_ALKnown_demographics, df_ALKnown_Echo, df_ALKnown_DescriptiveStats,
                    df_ALKnown_LabFitsLinear, df_ALKnown_LabFitsLinearWB, df_ALKnown_LabFitsQuadForce,
                    df_ALKnown_LabFitsExp2Force, df_ALKnown_MDNotesDescriptive, df_AL_known_tx_summary], axis =1 )

outputToExcel(df_data = df_ALKnown_summary, fileName_header= "AL-KnownPts", fileName_Common= "-labsAll-wFill-tx-Summary-One", fileName_suffix= outputSheet_Suffix,
        parent_dir= outputDir, folderName=folderNameOut, sheetName= "ALL")

df_ALOther_summary = pd.concat([df_ALOther_demographics, df_ALOther_Echo, df_ALOther_DescriptiveStats,
                    df_ALOther_LabFitsLinear, df_ALOther_LabFitsLinearWB, df_ALOther_LabFitsQuadForce,
                    df_ALOther_LabFitsExp2Force, df_ALOther_MDNotesDescriptive, df_AL_Unknown_tx_summary], axis =1 )

outputToExcel(df_data = df_ALOther_summary, fileName_header= "AL-UnknownPts", fileName_Common= "-labsAll-wFill-tx-Summary-One", fileName_suffix= outputSheet_Suffix,
        parent_dir= outputDir, folderName=folderNameOut, sheetName= "ALL")

In [46]:

# df_all = pd.concat([df_all_demographics, df_all_EchoTest, df_all_DescriptiveStats,
#                     df_all_LabFitsLinear, df_all_LabFitsLinearWB, df_all_LabFitsQuadForce,
#                     df_all_LabFitsExp2Force, df_all_LabFitsExp3Force], axis =1 )
# outputToExcel(df_data = df_all, fileName_header= "AllPts", fileName_Common= "-Summary-One", fileName_suffix= "_2024-06-28",
#         parent_dir= outputDir, folderName=folderNameOut, sheetName= "ALL")